In [1]:
import numpy as np, keras, os
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Activation,BatchNormalization,LeakyReLU,ZeroPadding2D,Input
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
data = ''# where the data is
models = ''# where to save the .hdf5
key = ''# which cqt to use

FOLDS = 5       # Number of folds for training

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [4]:
def initialize_model():
    
    model = Sequential()

    model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1,input_shape = x_train[0].shape))
    model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
    model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))

    model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
    model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(11))
    model.add(Activation('sigmoid'))
    
    return model

def han_model():
    
    x = Input(shape=(96,87,1))
    z = x
    
    for i in range(0,4):
        x = ZeroPadding2D((2,2))(x)
        x = Conv2D(2**(5+i),(3,3))(x) 
        x = LeakyReLU(0.33)(x)
        x = ZeroPadding2D((2,2))(x)
        x = Conv2D(2**(5+i),(3,3))(x)
        x = LeakyReLU(0.33)(x)
        if (i != 3):
            x = MaxPooling2D((3,3))(x)
            x = Dropout(0.25)(x)
    x = MaxPooling2D((9,9))(x)

    x = Flatten()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(0.33)(x)
    x = Dropout(0.5)(x)
    x = Dense(11,activation='sigmoid')(x)

    model = Model(input=z,output=x)
    
    return model

In [5]:
x, y = [], []
for i in range(FOLDS):
    x.append(np.load(data+'x_fold'+str(i)+'_'+key+'.npy'))
    y.append(np.load(data+'y_fold'+str(i)+'_'+key+'.npy'))

In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

In [4]:
for i in range(FOLDS):

    x_train = np.concatenate(tuple([x[k] for k in range(5) if not k==i]))
    y_train = np.concatenate(tuple([y[k] for k in range(5) if not k==i]))
    x_dev, y_dev = x[i], y[i]
    
    
    model = initialize_model(simple)
    #model = han_model()
    
    earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min',restore_best_weights=True)
    mcp_save = ModelCheckpoint(models+str(i)+'_'+key+'.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='min')
    
    model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    model.fit(x_train,y_train,epochs=200,batch_size=128,validation_data=(x_dev,y_dev),
             callbacks=[earlystop,mcp_save,reduce_lr_loss])
    
    print("Trained fold "+str(i))